In [7]:
from google.colab import drive
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Download stopwords if not already
nltk.download('stopwords')

# Load spacy model
nlp = spacy.load('en_core_web_sm')

drive.mount('/content/drive')

train = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/train.csv",low_memory=False)
train['crimeaditionalinfo'] = train['crimeaditionalinfo'].fillna("Null")
print(train.shape)

test = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/test.csv",low_memory=False)
test['crimeaditionalinfo'] = test['crimeaditionalinfo'].fillna("Null")
print(test.shape)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive
(93686, 12)
(31229, 12)


In [10]:
test_sc = test['sub_category'].fillna("null").unique()
train_sc = train['sub_category'].fillna("null").unique()

for c in train_sc:
  if c not in test_sc:
    print(c)

Against Interest of sovereignty or integrity of India


In [11]:
for c in test_sc:
  if c not in train_sc:
    print(c)

Computer Generated CSAM/CSEM
Cyber Blackmailing & Threatening
Sexual Harassment


In [6]:
subcategories = ['Child Pornography/Child Sexual Abuse Material (CSAM)',
 'Rape/Gang Rape-Sexually Abusive Content',
 'Sale, Publishing and Transmitting Obscene Material/Sexually Explicit Material',
 'Debit/Credit Card Fraud',
 'SIM Swap Fraud',
 'Internet Banking-Related Fraud',
 'Business Email Compromise/Email Takeover',
 'E-Wallet Related Frauds',
 'Fraud Call/Vishing',
 'Demat/Depository Fraud',
 'UPI-Related Frauds',
 'Aadhaar Enabled Payment System (AEPS) Fraud',
 'Email Phishing',
 'Cheating by Impersonation',
 'Fake/Impersonating Profile',
 'Profile Hacking/Identity Theft',
 'Provocative Speech of Unlawful Acts',
 'Impersonating Email',
 'Intimidating Email',
 'Online Job Fraud',
 'Online Matrimonial Fraud',
 'Cyber Bullying/Stalking/Sexting',
 'Email Hacking',
 'Damage to Computer Systems',
 'Tampering with Computer Source Documents',
 'Defacement/Hacking',
 'Unauthorized Access/Data Breach',
 'Online Cyber Trafficking',
 'Online Gambling/Betting Fraud',
 'Ransomware',
 'Cryptocurrency Crime',
 'Cyber Terrorism',
 'Any Other Cyber Crime',
 'Targeted scanning/probing of critical networks/systems.',
 'Compromise of critical systems/information.',
 'Unauthorised access to IT systems/data.',
 'Defacement of websites or unauthorized changes, such as inserting malicious code or ',
 'external links.',
 'Malicious code attacks (e.g., virus, worm, Trojan, Bots, Spyware, Ransomware, Crypto ',
 'miners).',
 'Attacks on servers (Database, Mail, DNS) and network devices (Routers).',
 'Identity theft, spoofing, and phishing attacks.',
 'Denial of Service (DoS) and Distributed Denial of Service (DDoS) attacks',
 'Attacks on critical infrastructure, SCADA, operational technology systems, and wireless ',
 'networks.',
 'Attacks on applications (e.g., E-Governance, E-Commerce).',
 'Data breaches.',
 'Data leaks.',
 'Attacks on Internet of Things (IoT) devices and associated systems, networks, and ',
 'servers.',
 'Attacks or incidents affecting digital payment systems.',
 'Attacks via malicious mobile apps.',
 'Fake mobile apps.',
 'Unauthorised access to social media accounts.',
 'Attacks or suspicious activities affecting cloud computing systems, servers, software, ',
 'and applications.',
 'Attacks or malicious/suspicious activities affecting systems related to Big Data, ',
 'Blockchain, virtual assets, and robotics.',
 'Attacks on systems related to Artificial Intelligence (AI) and Machine Learning (ML).',
 'Backdoor attacks.',
 'Disinformation or misinformation campaigns.',
 'Supply chain attacks.',
 'Cyber espionage.',
 'Zero-day exploits.',
 'Password attacks.',
 'Web application vulnerabilities.',
 'Hacking',
 'Malware attacks']

## Checking if phone numbers are present in text

In [ ]:
# Regex pattern to match Indian phone numbers
phone_number_pattern = r'\+?91[\s-]?\d{5}[\s-]?\d{5}|\b\d{10}\b'

# Function to find Indian phone numbers in text
def find_indian_phone_numbers(text):
    # Find all matching phone numbers
    phone_numbers = re.findall(phone_number_pattern, text)
    return phone_numbers

train['phone_numbers_detected'] = train['crimeaditionalinfo'].apply(lambda x: str(find_indian_phone_numbers(str(x))))
test['phone_numbers_detected'] = test['crimeaditionalinfo'].apply(lambda x: str(find_indian_phone_numbers(str(x))))

## Checking if email ids are present in text

In [ ]:
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
def find_emails(text):
    # Find all matching email addresses
    emails = re.findall(email_pattern, text)
    return emails

train['email_detected'] = train['crimeaditionalinfo'].apply(lambda x: str(find_emails(str(x))))
test['email_detected'] = test['crimeaditionalinfo'].apply(lambda x: str(find_emails(str(x))))

## Check for any numbers in the data

In [ ]:
number_pattern = r'\d+'
def find_numbers(text):
    # Find all matching numbers
    numbers = re.findall(number_pattern, text)
    return numbers

train['numbers_detected'] = train['crimeaditionalinfo'].apply(lambda x: str(find_numbers(str(x))))
test['numbers_detected'] = test['crimeaditionalinfo'].apply(lambda x: str(find_numbers(str(x))))

## Data Cleaning

In [126]:
# Define stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization using spacy
    doc = nlp(' '.join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]

    return ' '.join(lemmatized_tokens)

train['clean_msg'] = train['crimeaditionalinfo'].apply(lambda x: clean_text(str(x)))
print('done')
test['clean_msg'] = test['crimeaditionalinfo'].apply(lambda x: clean_text(str(x)))
print('done')

done
done


In [127]:
words = {
        'fruad': 'fraud',
        'financail': 'financial',
        'pls': 'please',
        'txn': 'transaction',
        'frezee': 'freeze',
        'frud': 'fraud',
        'finanacial': 'financial',
        'fraude': 'fraud'
    }
# Function to replace words in the text
def replace_words(text, word_dict):
    # Replace each word in the dictionary
    for incorrect_word, correct_word in word_dict.items():
        text = text.replace(incorrect_word, correct_word)
    return text

# Apply the function to the text column
train['cleaned_clean_msg'] = train['clean_msg'].apply(lambda x: replace_words(x.lower(), words))
test['cleaned_clean_msg'] = test['clean_msg'].apply(lambda x: replace_words(x.lower(), words))

In [ ]:
from textblob import TextBlob

text = "financail frezee fruad txn"

# Correct the spelling before lemmatization
corrected_text = str(TextBlob(text).correct())
print(corrected_text)

## Generate unigrams and bigrams

In [129]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [130]:
import pandas as pd
import nltk
from nltk.util import ngrams

# Function to generate unigrams and bigrams
def generate_ngrams(text, n):
    tokens = nltk.word_tokenize(text)  # Tokenize the text
    return list(ngrams(tokens, n))

# Apply the function to generate unigrams and bigrams
train['unigrams'] = train['cleaned_clean_msg'].apply(lambda x: generate_ngrams(x, 1))  # Unigrams
train['bigrams'] = train['cleaned_clean_msg'].apply(lambda x: generate_ngrams(x, 2))   # Bigrams

test['unigrams'] = test['cleaned_clean_msg'].apply(lambda x: generate_ngrams(x, 1))  # Unigrams
test['bigrams'] = test['cleaned_clean_msg'].apply(lambda x: generate_ngrams(x, 2))   # Bigrams

## Length of text

In [132]:
train['clean_text_length'] = train['cleaned_clean_msg'].apply(lambda x: len(str(x)))
test['clean_text_length'] = test['cleaned_clean_msg'].apply(lambda x: len(str(x)))

# train['clean_text_len_bins'],bins = pd.qcut(train['clean_text_length'], q=10,duplicates='drop')
# test['clean_text_len_bins'] = pd.cut(test['clean_text_length'], bins=bins,duplicates='drop')

In [134]:
train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/train.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/test.csv",index=False)

In [80]:
test = test.fillna("null")
test.groupby(['category','sub_category']).agg(
    count = ('crimeaditionalinfo', 'count'),
    # unique_count = ('cluster', 'nunique')
)

count
category                                           sub_category                                             
Any Other Cyber Crime                              Other                                                3670
Child Pornography CPChild Sexual Abuse Material... null                                                  123
Crime Against Women & Children                     Computer Generated CSAM/CSEM                            2
                                                   Cyber Blackmailing & Threatening                        1
                                                   Sexual Harassment                                       1
Cryptocurrency Crime                               Cryptocurrency Fraud                                  166
Cyber Attack/ Dependent Crimes                     Data Breach/Theft                                     171
                                                   Denial of Service (DoS)/Distributed Denial of S...    187
                                                   Hacking/Defacement                                    200
                                                   Malware Attack                                        170
                                                   Ransomware Attack                                     186
                                                   SQL Injection                                         167
                                                   Tampering with computer source documents              180
Cyber Terrorism                                    Cyber Terrorism                                        52
Hacking  Damage to computercomputer system etc     Damage to computer computer systems etc                39
                                                   Email Hacking                                         130
                                                   Tampering with computer source documents               14
                                                   Unauthorised AccessData Breach                        370
                                                   Website DefacementHacking                              39
Online Cyber Trafficking                           Online Trafficking                                     61
Online Financial Fraud                             Business Email CompromiseEmail Takeover                90
                                                   DebitCredit Card FraudSim Swap Fraud                 3556
                                                   DematDepository Fraud                                 222
                                                   EWallet Related Fraud                                1338
                                                   Fraud CallVishing                                    1827
                                                   Internet Banking Related Fraud                       2973
                                                   UPI Related Frauds                                   8890
Online Gambling  Betting                           Online Gambling  Betting                              134
Online and Social Media Related Crime              Cheating by Impersonation                             719
                                                   Cyber Bullying  Stalking  Sexting                    1366
                                                   EMail Phishing                                         54
                                                   FakeImpersonating Profile                             763
                                                   Impersonating Email                                    13
                                                   Intimidating Email                                     11
                                                   Online Job Fraud                                      294
                                                   Online Matrimonial 

## Clustering texts

In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Step 1: Preprocess and Transform Text Data
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()  # You can customize max_features

# Fit and transform the text data into TF-IDF matrix
X_tfidf = tfidf_vectorizer.fit_transform(train['clean_msg'].fillna("null"))

# Step 2: Apply DBSCAN Clustering
# Initialize DBSCAN
dbscan = DBSCAN(eps=0.25, min_samples=2, metric='cosine')  # eps is the distance threshold, tune for your data

# Fit DBSCAN model
dbscan.fit(X_tfidf)

# Get the cluster labels assigned by DBSCAN
train['cluster'] = dbscan.labels_

# Step 3: Evaluate clustering (optional, depends on the dataset)
# Since DBSCAN can assign some points to noise (-1), we'll only calculate silhouette score if there are more than 1 cluster
if len(set(dbscan.labels_)) > 1:
    silhouette_avg = silhouette_score(X_tfidf, dbscan.labels_)
    print(f"Silhouette Score: {silhouette_avg}")

# # You can further inspect each cluster
# for cluster_num in set(dbscan.labels_):
#     print(f"\nCluster {cluster_num}:")
#     print(train[train['cluster'] == cluster_num]['text'].values)

Silhouette Score: -0.02875997068922689


In [68]:
# train[train['cluster'] == 6]['clean_msg'].tolist()

In [70]:
train['cluster'].to_csv("/content/drive/MyDrive/IndiaAI_hackathon/train_with_more_data.csv",index=False)

In [66]:
train[train['cluster'] == 6]['sub_category'].value_counts()

,count
sub_category,


In [ ]:
train['cluster'].value_counts().reset_index().sort_values('count',ascending=False).head(20)

In [47]:
train[['clean_msg', 'cluster']]

,clean_msg,cluster
0,continue receive random call abusive message w...,-1
1,fraudster continuously message ask pay money s...,-1
2,act like police demand money add section text ...,-1
3,apna job apply job interview telecalle resourc...,-1
4,receive call lady state send new phone vivo re...,-1
...,...,...
93680,identity theft smishe sms fraud creditdebit ca...,571
93681,receive call number ask phone pay cash back of...,-1
93682,cyber stalk blackmailing phonesmsvoip call vic...,23
93683,call karke bola ki aapka lotary laga ha aru ac...,-1


## BOW

In [14]:
train[train['category'] == "Report Unlawful Content"]['crimeaditionalinfo'].tolist()

['The lady in the attached videos goes by the name Swathi Iyer on the social Media Platform ClubHouse \r\n\r\nHere she is seen sharing stage with people who abuse people just because they support the current government and PM Narendra Modi\r\n\r\nIn the video she is seen sharing stage with people who support the separatist movements of Punjab  Kashmir and Tamil Nadu\r\n\r\nThese people belong to the same group as the recently arrested Kashmiri separatist Mir Mushtaq\r\n\r\nRequest you to take cognizance and act accordingly']

In [7]:
train['category'].value_counts()

,count
category,
Online Financial Fraud,57434
Online and Social Media Related Crime,12140
Any Other Cyber Crime,10878
Cyber Attack/ Dependent Crimes,3608
RapeGang Rape RGRSexually Abusive Content,2822
Sexually Obscene material,1838
Hacking Damage to computercomputer system etc,1710
Sexually Explicit Act,1552
Cryptocurrency Crime,480


In [39]:
train = train[~train['category'].eq("Report Unlawful Content")].reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(train['clean_msg'].fillna("null"), train['category'].fillna("null"),
                                                    test_size=0.3, random_state=42, stratify=train['category'])

print(X_train.shape, X_test.shape)

(65579,) (28106,)


In [41]:
### Bag of Words (CountVectorizer)
# Initialize CountVectorizer for BoW
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)
train_bow = bow_vectorizer.transform(train['clean_msg'].fillna("null"))

# Train a Logistic Regression model with BoW features
bow_classifier = LogisticRegression()
bow_classifier.fit(X_train_bow, y_train)

# Make predictions on the test set
y_pred_bow = bow_classifier.predict(X_test_bow)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, log_loss

# Train
X_train['lr_model_score'] = bow_classifier.predict_proba(X_train_bow)[:, 1]
X_train['lr_model_prediction'] = bow_classifier.predict(X_train_bow)

train_metrics = {
    # "AUC": roc_auc_score(y_train, X_train['lr_model_score'], average='weighted'),
    "Accuracy": accuracy_score(y_train, X_train['lr_model_prediction']),
    "Precision": precision_score(y_train, X_train['lr_model_prediction'], average='weighted'),
    "Recall": recall_score(y_train, X_train['lr_model_prediction'], average='weighted'),
    "F1 Score": f1_score(y_train, X_train['lr_model_prediction'], average='weighted'),
    # "Log Loss": log_loss(y_train, X_train['lr_model_score'])
}

# Test
X_test['lr_model_score'] = bow_classifier.predict_proba(X_test_bow)[:, 1]
X_test['lr_model_prediction'] = bow_classifier.predict(X_test_bow)

test_metrics = {
    # "AUC": roc_auc_score(y_test, X_test['lr_model_score'], average='weighted'),
    "Accuracy": accuracy_score(y_test, X_test['lr_model_prediction']),
    "Precision": precision_score(y_test, X_test['lr_model_prediction'], average='weighted'),
    "Recall": recall_score(y_test, X_test['lr_model_prediction'], average='weighted'),
    "F1 Score": f1_score(y_test, X_test['lr_model_prediction'], average='weighted'),
    # "Log Loss": log_loss(y_test, X_test['lr_model_score'])
}

# Overall
train['lr_model_score'] = bow_classifier.predict_proba(train_bow)[:, 1]
train['lr_model_prediction'] = bow_classifier.predict(train_bow)

overall_train_metrics = {
    # "AUC": roc_auc_score(train['category'], train['lr_model_score'], average='weighted'),
    "Accuracy": accuracy_score(train['category'], train['lr_model_prediction']),
    "Precision": precision_score(train['category'], train['lr_model_prediction'], average='weighted'),
    "Recall": recall_score(train['category'], train['lr_model_prediction'], average='weighted'),
    "F1 Score": f1_score(train['category'], train['lr_model_prediction'], average='weighted'),
    # "Log Loss": log_loss(train['category'], train['lr_model_score'])
}

# OOT


# Create a DataFrame to compare metrics
metrics_df = pd.DataFrame({
    'Metric': list(train_metrics.keys()),
    'Train': list(train_metrics.values()),
    'Test': list(test_metrics.values()),
    'Overall Data': list(overall_train_metrics.values()),
    # 'OOT': list(oot_metrics1.values())
})

# Display the DataFrame
display(metrics_df)

,Metric,Train,Test,Overall Data
0,Accuracy,0.871544,0.741870,0.832641
1,Precision,0.867999,0.710076,0.822386
2,Recall,0.871544,0.741870,0.832641
3,F1 Score,0.864036,0.721977,0.821885


In [ ]:
# Evaluate the BoW model
print("Bag of Words Model Accuracy:", accuracy_score(y_test, y_pred_bow))
print("Classification Report for Bag of Words Model:\n", classification_report(y_test, y_pred_bow))

### TF-IDF (TfidfVectorizer)
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Logistic Regression model with TF-IDF features
tfidf_classifier = LogisticRegression()
tfidf_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred_tfidf = tfidf_classifier.predict(X_test_tfidf)

# Evaluate the TF-IDF model
print("TF-IDF Model Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report for TF-IDF Model:\n", classification_report(y_test, y_pred_tfidf))


## Thoughts

1. groupby category and unigrams and bigrams
2. We need classifier for category and sub_category classification
3. First target the low hanging fruits
4. Then target high hanging fruits - category level TF-IDF and others